# Sequential Chains

### Read sample data

In [4]:
import pandas as pd

In [5]:
reviews = pd.read_csv('data/reviews.csv')

In [32]:
reviews.head(10)

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


### Sequential chain

In [20]:
from langchain.chains import LLMChain, SequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [7]:
llm = ChatOpenAI(temperature=0.0)

In [34]:
prompt_1 = ChatPromptTemplate.from_template(
    '''
    Translate the following review to English:
    \n{review}
    '''
)
chain_1 = LLMChain(llm=llm,
                   prompt=prompt_1,
                   output_key='review_english')

In [35]:
prompt_2 = ChatPromptTemplate.from_template(
    '''
    Summarize the following review in one sentence:
    \n{review_english}
    '''
)
chain_2 = LLMChain(llm=llm,
                   prompt=prompt_2,
                   output_key='summary')

In [36]:
prompt_3 = ChatPromptTemplate.from_template(
    '''
    What language is the following review in?
    \n{review}
    '''
)
chain_3 = LLMChain(llm=llm,
                   prompt=prompt_3,
                   output_key='language')

In [37]:
prompt_4 = ChatPromptTemplate.from_template(
    '''
    Write a follow-up message to the following summary
    in the specified language:
    \Summary: {summary}
    \nLanguage: {language}
    '''
)
chain_4 = LLMChain(llm=llm,
                   prompt=prompt_4,
                   output_key='message')

In [44]:
prompt_5 = ChatPromptTemplate.from_template(
    '''
    Translate the follow-up message to English:
    \Message: {message}
    '''
)
chain_5 = LLMChain(llm=llm,
                   prompt=prompt_5,
                   output_key='response_english')

In [45]:
seq_chain = SequentialChain(chains=[chain_1, chain_2, chain_3, chain_4, chain_5],
                            input_variables=['review'],
                            output_variables=['review_english', 'summary', 'language', 'message', 'response_english'])

In [46]:
reviews.Review[5]

"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

In [47]:
seq_chain(reviews.Review[5])

{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'review_english': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the taste and foam of the product, suspecting that it may be an old batch or counterfeit.',
 'language': 'The following review is in French.',
 'message': "Cher(e) critique,\n\nNous avons bien pris en compte votre commentaire concernant notre produit. Nous sommes désolés d'apprendre que vous êtes déçu(e) par le goût et la mousse de celui-ci. Nous tenons à vous assurer que nous prenons vos préoccupations très au sérieux.\n\nNous comprenons votre inquiétude quant à la possibilité que le produit soit un lot ancien ou contrefait. Soyez assuré(e) que nous avons des procédures strictes en place pou